## Pay transparency data science round trip

In [1]:
from datetime import date, datetime
import numpy as np
from visier_api_data_out import QueryFilterDTO
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer

from visier_api import VisierApi
from model import Model

In [2]:
# file name must be the analytic object name, and in CSV format
FILE_PATH = './data/Employee.csv'
# This is the end time for the query
QUERY_END_TIME = date(2022, 5, 1)

visier_api: VisierApi = VisierApi()

# Prepare for the input data

To explore what data is available in a tenant, we could list properties with the `DataModelApi`.

In [4]:
schema = visier_api.list_data_model_properties(analytic_object='Employee').properties
list(filter(lambda x: x.id == "Employee.EmployeeID", schema))

UnauthorizedException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 24 Sep 2024 23:30:31 GMT', 'Content-Type': 'text/plain; charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'CF-Ray': '8c869012ba2b8393-SEA', 'CF-Cache-Status': 'DYNAMIC', 'Cache-Control': 'no-store, no-cache, must-revalidate, max-age=0', 'Set-Cookie': 'AWSALB=dPvl81wJGbRTK5Ij4cr6ZvfqklgZmXTKLLd52OcizwG7RFbkjht5Amawat6VuGiSDPI1SBFmsI9YzZbOjSREcTKIEGoqM1Ca3/c02txZxAQ/LEJp557ugMb9Kyht; Expires=Tue, 01 Oct 2024 23:30:31 GMT; Path=/, AWSALBCORS=dPvl81wJGbRTK5Ij4cr6ZvfqklgZmXTKLLd52OcizwG7RFbkjht5Amawat6VuGiSDPI1SBFmsI9YzZbOjSREcTKIEGoqM1Ca3/c02txZxAQ/LEJp557ugMb9Kyht; Expires=Tue, 01 Oct 2024 23:30:31 GMT; Path=/; SameSite=None; Secure, __cf_bm=rtTTkZxhAiBc9GfrvHKuh8PDU41ykS3TMJp3QTtABxo-1727220631-1.0.1.1-iXQSVcpxHGu5.o4qU2N5i8leLs08YqPWYyklRBGfhzp6iScd6SlRlifqpWAlJulXrZbo.WelWq3marLS_jP57Q; path=/; expires=Wed, 25-Sep-24 00:00:31 GMT; domain=.visierdev.io; HttpOn

Then we can compose a list query execution DTO for prediction analysis.

We query the attributes that we are interested as feature attributes:
- isFemale, Employee.Internal_Job_Experience, Highest_Education_Level_Achieved, Internal_Experience, External_Experience

and the training target attribute:
- Market_Direct_Compensation

In [6]:
# Fetch data from the public API 
execution = visier_api.compose_list_query_execution_dto(
    columns=[
        {"displayName": "EmployeeID", "attribute": "EmployeeID"},
        {"displayName": "IsWoman", "attribute": "isFemale"},
        {"displayName": "TimeInPosition", "attribute": "Employee.Internal_Job_Experience"},
        {"displayName": "Education", "attribute": "Highest_Education_Level_Achieved"},
        {"displayName": "OtherTenure", "attribute": "Internal_Experience"},
        {"displayName": "ExternalExperience", "attribute": "External_Experience"},
        {"displayName": "DirectCompensation", "attribute": "Market_Direct_Compensation"},
    ],
    end_time=datetime.strftime(QUERY_END_TIME, "%Y-%m-%d"),
    filters = [QueryFilterDTO(formula='Currency_Code="USD"')]
)

df = visier_api.download_data(execution)
df.head(5)

,FullName,EmployeeID,IsWoman,TimeInPosition,Education,OtherTenure,ExternalExperience,DirectCompensation,Currency
0,Renata Gutierrez,Employee-29191448,True,0,NaN,0,93,53506.95,USD
1,Alex Fernandez,Employee-121619,False,51,Bachelor,36,242,192529.79,USD
2,Brooks Lomas,Employee-13488561,False,13,NaN,12,78,162770.92,USD
3,Rick Kirby,Employee-25314844,False,5,NaN,0,119,93397.76,USD
4,Chaquille Bullock,Employee-22613614,False,10,NaN,0,132,64003.77,USD


Slice the dataframe for prediction model input(X) and output(y).

In [ ]:
PREDICTION_INPUT_COLUMNS = frozenset([
    'IsWoman', 
    'Education', 
    'TimeInPosition', 
    'OtherTenure', 
    'ExternalExperience'
])

X, y = df[PREDICTION_INPUT_COLUMNS], df['DirectCompensation']

X.shape, y.shape

# Regression model
Define regression model model

In [12]:
## define feature processing pipeline
features = FeatureUnion(
    [
        ("IsWoman", ColumnTransformer([
            ("one_hot",OneHotEncoder(
                handle_unknown="ignore", drop="first", dtype=np.float32
            ), ["IsWoman"])])
        ),
        ("Education", ColumnTransformer([
            ("one_hot",OneHotEncoder(
                handle_unknown="ignore", drop="first", dtype=np.float32
            ), ["Education"])])
        ),
        ("TimeInPosition", ColumnTransformer([
            ("scale",StandardScaler(),["TimeInPosition"])
        ])),
        ("OtherTenure", ColumnTransformer([
            ("scale",StandardScaler(),["OtherTenure"])
        ])),
        ("ExternalExperience", ColumnTransformer([
            ("scale",StandardScaler(),["ExternalExperience"])
        ])),
    ]
)

pipeline = Model.define_pipeline(features)
model = Model(pipeline=pipeline)

Pipeline(steps=[('feature_processing',
                 FeatureUnion(transformer_list=[('IsWoman',
                                                 ColumnTransformer(transformers=[('one_hot',
                                                                                  OneHotEncoder(drop='first',
                                                                                                dtype=<class 'numpy.float32'>,
                                                                                                handle_unknown='ignore'),
                                                                                  ['IsWoman'])])),
                                                ('Education',
                                                 ColumnTransformer(transformers=[('one_hot',
                                                                                  OneHotEncoder(drop='first',
                                                                                                dtype=<class 'numpy.float32'>,
                                                                                                handle_unknown='ig...
                                                 ColumnTransformer(transformers=[('scale',
                                                                                  StandardScaler(),
                                                                                  ['OtherTenure'])])),
                                                ('ExternalExperience',
                                                 ColumnTransformer(transformers=[('scale',
                                                                                  StandardScaler(),
                                                                                  ['ExternalExperience'])]))])),
                ('regressor',
                 TransformedTargetRegressor(regressor=LinearRegression(),
                                            transformer=FunctionTransformer(check_inverse=False,
                                                                            func=<ufunc 'log1p'>,
                                                                            inverse_func=<ufunc 'expm1'>)))])

# Model validation

Validate model and measure model performance

In [13]:
model.evaluate(X,y)

[CV] START .....................................................................
[CV] END  explained_variance: (train=0.282, test=0.337) neg_mean_absolute_error: (train=-36866.416, test=-38208.443) neg_mean_squared_error: (train=-5488562886.118, test=-6180789174.319) r2: (train=0.268, test=0.324) total time=   0.0s
[CV] START .....................................................................
[CV] END  explained_variance: (train=0.341, test=0.253) neg_mean_absolute_error: (train=-36357.616, test=-38828.778) neg_mean_squared_error: (train=-6194645638.773, test=-4367417226.424) r2: (train=0.329, test=0.229) total time=   0.0s
[CV] START .....................................................................
[CV] END  explained_variance: (train=0.324, test=0.292) neg_mean_absolute_error: (train=-38070.202, test=-35562.473) neg_mean_squared_error: (train=-5123962993.144, test=-6691398035.895) r2: (train=0.308, test=0.282) total time=   0.0s


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.0s


{'fit_time': np.float64(0.009197155634562174),
 'score_time': np.float64(0.004106998443603516),
 'test_r2': np.float64(0.2784647984753941),
 'train_r2': np.float64(0.3017514582634106),
 'test_neg_mean_squared_error': np.float64(-5746534812.212598),
 'train_neg_mean_squared_error': np.float64(-5602390506.011748),
 'test_neg_mean_absolute_error': np.float64(-37533.23137003673),
 'train_neg_mean_absolute_error': np.float64(-37098.078018933076),
 'test_explained_variance': np.float64(0.29403552177286335),
 'train_explained_variance': np.float64(0.3156941644581215)}

# Model training & inference
Fit model and view coefficients

In [14]:
model.fit(X,y)
model["regressor"].regressor_.coef_.astype(float)

array([-0.01533487,  0.00240708,  0.15020224,  0.12551257,  0.2097119 ,
        0.19642017,  0.07297258,  0.11740238,  0.18639227])

Use model for inference and evaluate the result

In [15]:
(predicted_value := model.predict(X))

array([ 72012.35851218, 141051.97734915,  79403.44676603, ...,
        72928.31933197,  75550.89870938,  70443.90819987])

Assign the result if it looks good

In [ ]:
df['Predicted'] = predicted_value

Save the model, so we can persist the coeffiencient and reproduce the results with the same input

In [16]:
model.save()

# To load the model, simply:
# model2 = Model()
# model2.load('data/model.2024-09-01.pkl')
#
# And to reproduce the result:
# model2.pipeline.predict(X)

# Save Output

Get the project schema from the tenant

In [39]:
(schema_columns := [
    s.name
    for s in visier_api.list_project_schema()
    if s.is_mandatory or "Market_Direct_Compensation" in s.name
])

['EmployeeID', 'Employee_Burnout_Index', 'EventDate']

Message the prediction data so it matches the tenant's schema requirements.

In [ ]:
# Convert the dataframe to the expected format
CURRENT_TIME = datetime.now()
output = df[['EmployeeID', 'Predicted']]
output['EventDate'] = datetime.strftime(datetime(2022,7,1), "%Y-%m-%d")
# output['regressionRanDate'] = CURRENT_TIME
output.rename(columns={
    'EmployeeID': 'EmployeeID',
    'EventDate': 'EventDate',
    'Predicted': 'Market_Direct_Compensation'
}, inplace=True)
assert set(output.columns) == set(schema_columns)

# Save the output
output.to_csv(FILE_PATH, index=False)

# Upload the result to the application

In [41]:
tx_id = visier_api.upload_data(FILE_PATH, analytic_object_name='Employee')

Transaction status: SUCCEEDED, message: ReceivingDataJob scheduled
